In [1]:
import json
import requests

In [2]:
url = "http://127.0.0.1:4050/ingest-files"

payload = json.dumps({
    "bucket_name": "rag-accelerator-2.0",
    "connection_name": "milvus_connect",
    "directory": "watsonx_docs",
    "index_name": "sample"
})

response = requests.request("POST", url, data=payload)

print(response.text)

{"status":"success","message":"Data Ingestion successful for 10 document chunks into vector database!"}


In [3]:
url = "http://127.0.0.1:4050/query"

payload = json.dumps({
    "connection_name": "milvus_connect",
    "index_name": "sample",
    "query": "How to perform decision optimization?"
})

response = requests.request("POST", url, data=payload)

print(response.text)

{"data":{"answer":[{"id":null,"metadata":{"title":"Decision Optimization migration","source":"watsonx_docs/migration.html","document_url":"https://www.ibm.com/docs/SSYOK8/DO/WML_Deployment/migration.html","page_number":"","chunk_seq":1,"id":"8d0ccd0a187f50b2f19fde4daff510e4f85a8ecb21aef5315ef97f1748dd652e"},"page_content":"Document Title: Decision Optimization migration\n Document Content: Decision Optimization migration\n\n\n\nMigrating\nYou can find different migration guides in this section.\n\n\n\nMigrating from COS or DB2 to a connection asset\nYou can use a connection asset to access COS or DB2 data assets. \nMigrating from Watson Machine Learning API V4 Beta\nAs the Watson Machine Learning V4 API now released, if you previously used the Watson Machine Learning  V4 Beta API for Decision Optimization, read here about some changes and improvements.\nMigrating Python code for Decision Optimization with Machine Learning-v2 instances\nUse the latest version of the Watson Machine Learn

In [ ]:
# QnA Queries
url = "http://127.0.0.1:4050/ai/qna/query"

payload = json.dumps({
    "question" : "how to perform decision optimization?"   
})

response = requests.request("POST", url, data=payload)

print(response.text)

{"answer":{"response":"To perform decision optimization, you need to follow a structured approach. First, define the decision problem and identify the key objectives, constraints, and variables involved. Next, gather relevant data and build a mathematical model that represents the decision-making process. This model can be a linear or nonlinear program, depending on the complexity of the problem. Then, use optimization algorithms, such as linear programming or mixed-integer programming, to find the optimal solution. You can utilize software tools like CPLEX, Gurobi, or MATLAB to solve the optimization model. Finally, analyze the results, validate the solution, and implement the optimal decision. It's also essential to consider uncertainty and sensitivity analysis to ensure the robustness of the solution. By following these steps, you can make informed, data-driven decisions that maximize value and minimize risk.","Hallucination Detection":"NA"},"documents":{},"expert_answer":"No expert

In [1]:
from dotenv import load_dotenv
import os

from ibm_watsonx_gov.config import Credentials
from ibm_watsonx_gov.entities.enums import Region

from ibm_watsonx_gov.config import GenAIConfiguration
from ibm_watsonx_gov.metrics import ContextRelevanceMetric, FaithfulnessMetric, AnswerSimilarityMetric, AnswerRelevanceMetric
from ibm_watsonx_gov.entities.enums import MetricGroup

load_dotenv()

question_field = "question"
context_field = "contexts"

config = GenAIConfiguration(
    input_fields=[question_field],
    context_fields=[context_field],
    output_fields=["answer"],
    reference_fields=["ground_truth"]
)

metrics = [
    ContextRelevanceMetric(),
    FaithfulnessMetric(),
    AnswerSimilarityMetric(),
    AnswerRelevanceMetric(),
]

metric_groups = [
    MetricGroup.RETRIEVAL_QUALITY,
    MetricGroup.ANSWER_QUALITY,
    MetricGroup.CONTENT_SAFETY
]

In [9]:
from ibm_watsonx_gov.clients.api_client import APIClient
from ibm_watsonx_gov.evaluators import MetricsEvaluator

credentials = Credentials(
    api_key=os.getenv("IBM_API_KEY"),
    service_instance_id=os.getenv("WXG_SERVICE_INSTANCE_ID"),
)
evaluator = MetricsEvaluator(api_client = APIClient(credentials=credentials), configuration=config)

In [ ]:
evaluation_result = evaluator.evaluate(
    data=input_df,
    metrics=metrics,
    metric_groups=metric_groups
)